In [12]:
# ライブラリ読み込み
import numpy as np
from decimal import Decimal, InvalidOperation

関数の読み込み

In [13]:
def _infer_original_resolution(values, max_decimal_places=12):
    """
    値の性質から「元の刻み幅」を推定する。
    - すべて整数（整数として表せる）なら 1
    - それ以外は、小数点以下の最大桁数から 10^-k を刻み幅として返す
      (例: 1.23, 2.5 -> max 2桁 => 0.01)
    """
    # Decimalで安全に桁数を見るため、文字列化して扱う
    decimals = []
    for v in values:
        s = str(v)
        try:
            d = Decimal(s)
        except InvalidOperation:
            # 万一変換できない場合は float にフォールバック
            d = Decimal(str(float(v)))
        decimals.append(d)

    # 整数判定（小数部分が0か）
    if all(d == d.to_integral_value() for d in decimals):
        return 1.0

    # 小数点以下の桁数（指数）を見て最大桁を推定
    # Decimal('1.2300') は exponent=-4 になりうるので正規化して桁数を安定化
    max_places = 0
    for d in decimals:
        dn = d.normalize()
        exp = -dn.as_tuple().exponent  # 小数点以下桁数
        if exp > max_places:
            max_places = exp

    # 過剰に細かく推定しないための上限
    max_places = min(max_places, max_decimal_places)

    return float(Decimal(1) / (Decimal(10) ** max_places))


def analyze_and_normalize(data):
    """
    data: list or np.ndarray of numerical values

    Returns (dict):
      - min
      - max
      - normalized (0-1)
      - original_resolution (元の刻み幅: 自動推定)
      - normalized_resolution (正規化後の分解能 = original_resolution / (max-min))
    """
    data = np.asarray(data, dtype=float)

    data_min = float(np.min(data))
    data_max = float(np.max(data))
    data_range = data_max - data_min

    if data_range == 0:
        raise ValueError("全て同じ値のため、0-1正規化できません（max-min=0）")

    normalized = (data - data_min) / data_range

    # 元の刻み幅（自動推定）
    original_resolution = _infer_original_resolution(data)

    # 正規化後の分解能
    normalized_resolution = original_resolution / data_range

    return {
        "min": data_min,
        "max": data_max,
        "normalized": normalized,
        "original_resolution": original_resolution,
        "normalized_resolution": normalized_resolution,
    }

In [16]:
data = [223455, 283445, 323455]

result = analyze_and_normalize(data)

for k, v in result.items():
    print(f"{k}: {v}")


min: 223455.0
max: 323455.0
normalized: [0.     0.5999 1.    ]
original_resolution: 1.0
normalized_resolution: 1e-05


In [18]:
data = [1225, 657, 225]

result = analyze_and_normalize(data)

for k, v in result.items():
    print(f"{k}: {v}")


min: 225.0
max: 1225.0
normalized: [1.    0.432 0.   ]
original_resolution: 1.0
normalized_resolution: 0.001


In [19]:
data = [12, 14, 22]

result = analyze_and_normalize(data)

for k, v in result.items():
    print(f"{k}: {v}")


min: 12.0
max: 22.0
normalized: [0.  0.2 1. ]
original_resolution: 1.0
normalized_resolution: 0.1
